План:
1. Очистить набор данных, полученных на предыдущих этапах.
2. Выбрать метод и выполнить нормализацию


In [1]:
import pandas as pd
import numpy as np

In [118]:
df = pd.read_csv('dataset_with_descriptors.csv')


/tmp/ipykernel_295056/1746065682.py:1: DtypeWarning: Columns (242,280,281,282,283,284,285,286,287,288,364,365,366,373,374,375,379,380,381,382,383,384,391,392,393,400,401,402,409,410,411,418,419,420,427,428,429,436,437,438,445,446,447,454,455,456,487,488,489,490,491,492,493,494,495,571,572,573,580,581,582,589,590,591,595,596,597,598,599,600,607,608,609,616,617,618,625,626,627,634,635,636,643,644,645,652,653,654,661,662,663,670,671,672,678,679,680,684,685,686,687,688,692,693,694,695,696,700,701,702,703,704,710,711,712,718,719,720,726,727,728,734,735,736,742,743,744,750,751,752,758,759,760,766,767,768,774,775,776,780,781,782,783,784,788,789,790,791,792,796,797,798,799,800,806,807,808,814,815,816,822,823,824,830,831,832,838,839,840,846,847,848,854,855,856,862,863,864,874,875,905,918,931,944,957,970,983,996,1018,1044,1052,1053,1054,1055,1056,1057,1058,1060,1068,1069,1070,1071,1072,1073,1074,1117,1460,1461,1462,1463,1526,1527,1528,1779,1807,1808,1810,1838) have mixed types. Specify dtype opt

В лекции описан следующий план очистки  
1. STEPS OF DATA CLEANING  
2. Data inspection  
3. Missing data handling  
4. Outlier detection  
5. Data transformation Duplicates handling  
6. Data normalization


# Data inspection

Пропущенных значений не выявлено, но это не значит, что их нет :)

In [119]:
df_missing = df.isnull().sum() # серия с кол-вом пропусков по колонкам
missing_cols = df_missing[df_missing > 0].index # индексы колонок с пропусками

print('Колонки с пропущенными значениями:')
print(missing_cols)

Колонки с пропущенными значениями:
Index([], dtype='object')


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20004 entries, 0 to 20003
Columns: 1840 entries, mol_id to mZagreb2
dtypes: bool(2), float64(857), int64(427), object(554)
memory usage: 280.6+ MB


### Видим, что довольно много столбцов типа object. object(554)
В черновом варианте кода я вычислил, что некоторые неподгруженные данные заполнялись строкой с сообщением об ошибке.  
Каждое такое значение начинается на одно из этих слов: ['float', 'invalid', 'min()', 'max()', 'module'].  
Далее я покажу как работал с такими значениями.

Создаём датасет из столбцов типа object, ведь в них вероятнее всего есть ошибки.  
Далее выводим кол-во значений содержащих одно из слов ['float', 'invalid', 'min()', 'max()', 'module'].  
Это нужно сделать, чтобы потом решить стоит удалять весь столбец если ошибок много или строчки если их мало

In [121]:
def count_phrases_in_columns(df, phrases):
    # Выбор столбцов типа 'object' из исходного датафрейма
    df_cat = df.select_dtypes(include=['object'])

    for phrase in phrases:
        # Инициализация словаря для текущей фразы
        phrase_counts = {}

        # Подсчет количества встречающихся фраз в каждом столбце
        for col in df_cat.columns:
            count = df_cat[col].astype(str).apply(lambda x: x.count(phrase)).sum()
            if count != 0:
                phrase_counts[col] = count

        # Вывод результатов для текущей фразы
        print(f"Фраза '{phrase}':")
        for col, count in phrase_counts.items():
            print(f" '{col}': {count} ")

# Фразы для подсчета
phrases = ['float', 'invalid', 'min()', 'max()', 'module']

# Вызов функции с передачей датафрейма и фраз
count_phrases_in_columns(df, phrases)

Фраза 'float':
 'AXp-1d': 2 
 'AXp-2d': 3 
 'AXp-3d': 7 
 'AXp-4d': 26 
 'AXp-5d': 350 
 'AXp-6d': 1641 
 'AXp-7d': 5634 
 'AXp-1dv': 2 
 'AXp-2dv': 3 
 'AXp-3dv': 7 
 'AXp-4dv': 26 
 'AXp-5dv': 350 
 'AXp-6dv': 1641 
 'AXp-7dv': 5634 
 'Kier1': 2 
 'Kier2': 3 
 'Kier3': 7 
 'MDEC-11': 15026 
 'MDEC-12': 7724 
 'MDEC-13': 7905 
 'MDEC-14': 14263 
 'MDEC-22': 3931 
 'MDEC-23': 1985 
 'MDEC-24': 12234 
 'MDEC-33': 5498 
 'MDEC-34': 12870 
 'MDEC-44': 18763 
 'MDEO-11': 16459 
 'MDEO-12': 14962 
 'MDEO-22': 18145 
 'MDEN-11': 19153 
 'MDEN-12': 17602 
 'MDEN-13': 19143 
 'MDEN-22': 17585 
 'MDEN-23': 18933 
 'MDEN-33': 19951 
 'RotRatio': 2 
Фраза 'invalid':
 'AATS3dv': 3 
 'AATS4dv': 3 
 'AATS5dv': 78 
 'AATS6dv': 3130 
 'AATS7dv': 12305 
 'AATS8dv': 18031 
 'AATS3d': 3 
 'AATS4d': 3 
 'AATS5d': 78 
 'AATS6d': 3130 
 'AATS7d': 12305 
 'AATS8d': 18031 
 'AATS3s': 1 
 'AATS4s': 1 
 'AATS5s': 76 
 'AATS6s': 3128 
 'AATS7s': 12303 
 'AATS8s': 18029 
 'AATS3Z': 3 
 'AATS4Z': 3 
 'AATS5Z': 78 

Далее пишем код, который:
1. Снова выбирает столбцы типа 'object' из датафрейма df и сохраняет их в df_cat.  
2. Подсчитывает количество встречающихся ошибок в каждом столбце и сохраняет результаты в словаре float_counts.  
3. Удаляет столбцы, в которых количество встречающихся ошибок больше 2000 (10% от датасета).  
4. Если ошибок меньше 10%, то удаляем строки с ними, Датасет не сильно пострадает от этого, поскольку выборка довольно большая.
5. В конце в идеале останутся только числовые значения и мы пробуем конвертировать столбец в тип FLOAT, если все слова не удалены, словим ошибку.

In [122]:
# Выбор столбцов типа 'object' из исходного датафрейма, оставляем 'SMILES'
df_cat = df.select_dtypes(include=['object']).drop(columns='smiles')

# Удаление столбцов и строк по условиям
phrases = ['float', 'invalid', 'min()', 'max()', 'module', 'there']
for phrase in phrases:
    # Удаление столбцов
    columns_to_remove = df_cat.columns[df_cat.apply(lambda x: x.astype(str).str.count(phrase).sum() > 2000)].tolist()
    df = df.drop(columns=columns_to_remove)
    print(f"Удалены столбцы с фразой '{phrase}': {columns_to_remove}")

    # Удаление строк
    condition = df.apply(lambda row: any(phrase in str(value) for value in row), axis=1)
    df = df[~condition]
    print(f"Удалено строк с фразой '{phrase}': {condition.sum()}")

# Преобразование столбцов 'object' в тип float, кроме 'SMILES'
for column in df.select_dtypes(include=['object']).columns:
    if column != 'smiles':
        try:
            df[column] = pd.to_numeric(df[column], errors='coerce')
        except ValueError:
            print(f"Столбец {column} не может быть преобразован в тип float.")

# Печать информации о количестве удаленных строк
print(f"Удалено строк: {condition.sum()}")
print(df.dtypes.value_counts())

Удалены столбцы с фразой 'float': ['AXp-7d', 'AXp-7dv', 'MDEC-11', 'MDEC-12', 'MDEC-13', 'MDEC-14', 'MDEC-22', 'MDEC-24', 'MDEC-33', 'MDEC-34', 'MDEC-44', 'MDEO-11', 'MDEO-12', 'MDEO-22', 'MDEN-11', 'MDEN-12', 'MDEN-13', 'MDEN-22', 'MDEN-23', 'MDEN-33']
Удалено строк с фразой 'float': 3177
Удалены столбцы с фразой 'invalid': ['AATS6dv', 'AATS7dv', 'AATS8dv', 'AATS6d', 'AATS7d', 'AATS8d', 'AATS6s', 'AATS7s', 'AATS8s', 'AATS6Z', 'AATS7Z', 'AATS8Z', 'AATS6m', 'AATS7m', 'AATS8m', 'AATS6v', 'AATS7v', 'AATS8v', 'AATS6se', 'AATS7se', 'AATS8se', 'AATS6pe', 'AATS7pe', 'AATS8pe', 'AATS6are', 'AATS7are', 'AATS8are', 'AATS6p', 'AATS7p', 'AATS8p', 'AATS6i', 'AATS7i', 'AATS8i', 'AATSC6c', 'AATSC7c', 'AATSC8c', 'AATSC6dv', 'AATSC7dv', 'AATSC8dv', 'AATSC6d', 'AATSC7d', 'AATSC8d', 'AATSC6s', 'AATSC7s', 'AATSC8s', 'AATSC6Z', 'AATSC7Z', 'AATSC8Z', 'AATSC6m', 'AATSC7m', 'AATSC8m', 'AATSC6v', 'AATSC7v', 'AATSC8v', 'AATSC6se', 'AATSC7se', 'AATSC8se', 'AATSC6pe', 'AATSC7pe', 'AATSC8pe', 'AATSC6are', 'AATSC7a

Второй раз вызываем функцию проверяющую кол-во ошибок в столбцах.  
Ожидаемое поведение: Пустой вывод.

In [123]:
# Вызов функции с передачей датафрейма и фраз
count_phrases_in_columns(df, phrases)

Фраза 'float':
Фраза 'invalid':
Фраза 'min()':
Фраза 'max()':
Фраза 'module':
Фраза 'there':


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16804 entries, 0 to 20003
Columns: 1507 entries, mol_id to mZagreb2
dtypes: bool(2), float64(1077), int64(427), object(1)
memory usage: 193.1+ MB


In [126]:
# Сохранение в CSV-файл
df.to_csv('dataset_with_descriptors_without_errors_v2.0.csv', index=False)

#### Видим что остался только 1 столбец типа object - smiles.   
#### Значит удалить ошибки у нас получилось!  
#### Этап Data inspection завершён.  

# Missing data handling

Как я говорил ранее пропущенные значения умеют маскироваться.  
Пропущенные значения в виде ошибок мы удалили, теперь есть подозрение что пропущенные ошибки могут выглядеть просто как "0"
Будем работать с этим.  

In [127]:
df = pd.read_csv('dataset_with_descriptors_without_errors_v2.0.csv')

Значение 0 может иметь несколько смыслов.  
0 - как числовое значение в числовом показателе.  
0 - как категориальное значение указывающее на принадлежность к определённой категории.  
0 - как пропущенное значение.
Все эти случаи объединяет одно. Если таких значений очень много, то смысла от такого столбца немного.  
Я решил, что если 75% столбца состоит из значения 0, то такой столбец нам не нужен.

In [128]:
# Создаем словарь для хранения количества нулей в каждом столбце и их названий
zero_counts = {}

# Проходимся по каждому столбцу DataFrame
for col in df.columns:
    # Подсчитываем количество нулей в столбце
    zero_count = (df[col] == 0).sum()
    # Добавляем результат в словарь
    zero_counts[col] = zero_count

# Создаем новый DataFrame, оставляя только те столбцы, в которых количество нулей не превышает 15000
df = df.loc[:, [col for col in df.columns if zero_counts[col] <= 15000]]

# Выводим названия удаленных столбцов и их количество
removed_cols = [col for col in zero_counts if zero_counts[col] > 15000]
print(f"Удалено {len(removed_cols)} столбцов: {', '.join(removed_cols)}")

Удалено 382 столбцов: NumRadicalElectrons, PEOE_VSA13, PEOE_VSA14, SMR_VSA8, SlogP_VSA10, SlogP_VSA11, SlogP_VSA12, SlogP_VSA7, SlogP_VSA8, SlogP_VSA9, EState_VSA11, VSA_EState10, NumAromaticCarbocycles, fr_Al_COO, fr_ArN, fr_Ar_COO, fr_Ar_NH, fr_Ar_OH, fr_COO, fr_COO2, fr_C_S, fr_HOCCN, fr_Imine, fr_NH2, fr_N_O, fr_Ndealkylation1, fr_Ndealkylation2, fr_Nhpyrrole, fr_SH, fr_aldehyde, fr_alkyl_carbamate, fr_alkyl_halide, fr_allylic_oxid, fr_amide, fr_amidine, fr_aniline, fr_aryl_methyl, fr_azide, fr_azo, fr_barbitur, fr_benzene, fr_benzodiazepine, fr_diazo, fr_dihydropyridine, fr_epoxide, fr_ester, fr_furan, fr_guanido, fr_halogen, fr_hdrzine, fr_hdrzone, fr_imidazole, fr_imide, fr_isocyan, fr_isothiocyan, fr_ketone_Topliss, fr_lactam, fr_lactone, fr_methoxy, fr_morpholine, fr_nitro, fr_nitro_arom, fr_nitro_arom_nonortho, fr_nitroso, fr_oxazole, fr_oxime, fr_para_hydroxylation, fr_phenol, fr_phenol_noOrthoHbond, fr_phos_acid, fr_phos_ester, fr_piperdine, fr_piperzine, fr_priamide, fr_pr

Далее можно посмотреть кол-во столбцов, где уникальных значений очень мало,  
вероятно они тоже могут содержать большой кусок проопущенных значений.

In [129]:
unique_counts = df.nunique().sort_values()  
few_uniques = unique_counts[unique_counts < 10].index

print('Колонки с менее чем 10 уникальными значениями:')

for col in few_uniques:

    unique_val_count = unique_counts[col] 
    print(f'{col}: {unique_val_count} уникальных значений')

    vc = df[col].value_counts()
    
    print(f'{col}:')
    for unique_val, count in vc.items():
        print(f' - {unique_val}: {count} ({round(100 * count / len(df), 2)}%)')

    print()

Колонки с менее чем 10 уникальными значениями:
mol_id: 0 уникальных значений
mol_id:

Lipinski: 1 уникальных значений
Lipinski:
 - True: 16804 (100.0%)

n7FARing: 2 уникальных значений
n7FARing:
 - 0: 14821 (88.2%)
 - 1: 1983 (11.8%)

n6FARing: 2 уникальных значений
n6FARing:
 - 0: 14627 (87.04%)
 - 1: 2177 (12.96%)

n7FRing: 2 уникальных значений
n7FRing:
 - 0: 14821 (88.2%)
 - 1: 1983 (11.8%)

n6FRing: 2 уникальных значений
n6FRing:
 - 0: 14627 (87.04%)
 - 1: 2177 (12.96%)

nFARing: 3 уникальных значений
nFARing:
 - 0: 9069 (53.97%)
 - 1: 7726 (45.98%)
 - 2: 9 (0.05%)

nFHRing: 3 уникальных значений
nFHRing:
 - 0: 10598 (63.07%)
 - 1: 6202 (36.91%)
 - 2: 4 (0.02%)

nFRing: 3 уникальных значений
nFRing:
 - 0: 8918 (53.07%)
 - 1: 7877 (46.88%)
 - 2: 9 (0.05%)

nHeavyAtom: 3 уникальных значений
nHeavyAtom:
 - 9: 14549 (86.58%)
 - 8: 2049 (12.19%)
 - 7: 206 (1.23%)

n5aHRing: 3 уникальных значений
n5aHRing:
 - 0: 14476 (86.15%)
 - 1: 2220 (13.21%)
 - 2: 108 (0.64%)

naHRing: 3 уникальных

Видим много столбцов дублирющих друг друга, удалим их при анализе корреляций.
Столбцы mol_id и Lipinski абсолютно точно бесполезны - удаляем. 
Также в ходе выполнения работы я иногда возвращаюсь назад.  
столбцы homo и lumo - столбцы которые напрямую зависят от gap, грубо говоря это также целевые переменные.  
Поскольку использовать для обучения мы их не можем - дропаем.


In [130]:
to_drop = ['mol_id', 'Lipinski', 'homo', 'lumo'] 
df.drop(columns=to_drop, inplace=True)

На этом Missing data handling закончен.

In [131]:
# Сохранение в CSV-файл
df.to_csv('dataset_with_descriptors_without_errors_v2.1.csv', index=False)

# Outlier detection

In [132]:
df = pd.read_csv('dataset_with_descriptors_without_errors_v2.1.csv')

In [133]:
df

,smiles,A,B,C,mu,alpha,gap,r2,zpve,u0,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,N=C1OCC2=C1ON=C2,3.42254,1.67692,1.13352,4.5643,67.77,0.2048,972.8206,0.091971,-452.735916,...,9.030137,54.421098,124.027277,9.540560,77,9,48.0,57.0,2.583333,1.972222
1,O=C1CCC11C2CCN12,3.00351,1.58602,1.24538,1.9055,77.52,0.2091,1039.1092,0.147846,-401.909060,...,10.358473,64.099976,123.068414,6.837134,78,9,60.0,82.0,2.395833,1.819444
2,O=CC1CCC2OCC12,3.11416,1.26756,1.03765,1.6498,75.15,0.2215,1166.1361,0.160480,-423.030316,...,9.399389,53.862370,126.068080,6.635162,82,9,48.0,58.0,2.583333,2.055556
3,C1C2C3N=C4OC1C4C23,3.12126,2.32028,1.94917,3.6237,69.65,0.2066,763.5215,0.127690,-400.695008,...,10.316094,64.041419,121.052764,7.565798,68,8,66.0,90.0,1.416667,1.666667
4,CC1(O)C2CC1(O2)C#C,2.37047,1.54170,1.35971,1.7895,76.02,0.2569,1039.3059,0.133262,-421.725719,...,10.682377,42.876740,124.052429,7.297202,78,10,56.0,74.0,3.986111,1.854167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16799,c1coc2c1C(=O)CC2,3.17605,1.63047,1.09187,3.0955,73.12,0.2127,1033.0148,0.115102,-420.687271,...,9.030137,54.421098,122.036779,8.135785,77,9,48.0,57.0,2.583333,1.972222
16800,CC1CN1C(=N)C#N,3.12217,1.77668,1.34512,3.1791,69.13,0.2075,927.9641,0.118744,-358.830448,...,8.885579,56.024874,109.063997,7.270933,66,7,38.0,44.0,3.833333,1.888889
16801,COC1CC(=N)OC1=O,3.40512,1.21702,0.92748,2.1467,67.72,0.2416,1215.9934,0.123752,-475.063357,...,8.720787,51.566550,129.042593,8.065162,85,9,42.0,47.0,4.083333,2.111111
16802,OC1CC11OCC1O,3.22232,1.74690,1.53428,1.4181,60.90,0.2937,882.3828,0.129128,-420.823950,...,9.889845,60.114861,116.047344,7.252959,58,7,48.0,62.0,3.034722,1.666667


Нам нужно найти выбросы в числовых столбцах. Их очень много поэтому применяем IQR к каждому,  
и смотрим сколько значений в каждом столбце выходят за границы квартилей.

In [134]:
# Находим числовые колонки  
numeric_columns = list(df.select_dtypes(include=['integer', 'float']).columns)

outliers_per_column = {}
threshold = 3   

# Для каждой числовой колонки  
for column in numeric_columns:

  # Вычисляем границы  
  q1 = df[column].quantile(0.25)
  q3 = df[column].quantile(0.75)  
  iqr = q3 - q1
  
  lower_bound = q1 - (threshold * iqr)
  upper_bound = q3 + (threshold * iqr)
   
  # Выявляем выбросы
  outliers = ((df[column] < lower_bound) | (df[column] > upper_bound)).sum()

  outliers_per_column[column] = outliers

# Конвертируем в список и сортируем
outliers_list = list(outliers_per_column.items())
outliers_list.sort(key=lambda x: x[1], reverse=True)

print('Колонки с выбросами:')  
for column, outliers in outliers_list:
  if outliers > 0:
      print(f'{column}: {outliers} выбросов')

Колонки с выбросами:
SssssC: 6269 выбросов
SdssC: 5874 выбросов
ATSC7s: 4914 выбросов
n4HRing: 3968 выбросов
n4AHRing: 3968 выбросов
ATSC7se: 3881 выбросов
nBondsT: 3792 выбросов
NtsC: 3792 выбросов
StsC: 3792 выбросов
n3HRing: 3614 выбросов
n3AHRing: 3614 выбросов
fr_Al_OH_noTert: 3584 выбросов
ATSC7c: 3436 выбросов
ATSC7are: 3432 выбросов
EState_VSA1: 3414 выбросов
EState_VSA1.1: 3414 выбросов
C1SP1: 3388 выбросов
n5AHRing: 3359 выбросов
nAromAtom: 3273 выбросов
nAromBond: 3273 выбросов
nBondsA: 3273 выбросов
naRing: 3273 выбросов
NumAromaticRings: 3255 выбросов
naHRing: 3242 выбросов
NumAromaticHeterocycles: 3224 выбросов
BCUTv-1l: 3216 выбросов
ATSC7pe: 3146 выбросов
SMR_VSA2: 3145 выбросов
SMR_VSA2.1: 3145 выбросов
ATS6dv: 3144 выбросов
NaasC: 3036 выбросов
SaasC: 3036 выбросов
PEOE_VSA4: 2942 выбросов
PEOE_VSA4.1: 2942 выбросов
fr_Ar_N: 2936 выбросов
NssNH: 2933 выбросов
SssNH: 2933 выбросов
NaaCH: 2869 выбросов
SaaCH: 2869 выбросов
PEOE_VSA12: 2748 выбросов
PEOE_VSA12.1: 2748 вы

Видим что более 20% столбцов имеют от 2000 выбросов. Способы решения проблемы:  
2.	Правило 3 сигм - выявление точек, отстоящих от среднего больше, чем на 3 стандартных отклонения.  
3.	Методы машинного обучения - специальные алгоритмы (типа изоляционного леса) для автоматического поиска аномалий.  
4.	Удаление выбросов - простой, но не всегда подходящий способ.  
5.	Замена на предельные значения - замена экстремальных значений на максимум/минимум диапазона.  


После часа раздумий и дискуссий был сделан вывод:  
Раз мы имеем дело с дескрипторами молекул, полученными на основе SMILES,  
то наличие выбросов может быть обоснованным и не являться шумом или аномалией.  
Поэтому выбросы будем искать только по столбцам оригинального датасета.  

In [135]:
# Список интересующих числовых колонок  
num_cols = ['A', 'B', 'C', 'mu', 'alpha', 'gap', 
            'r2', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv']

outliers_per_column = {}
threshold = 3

# Перебираем только нужные колонки
for column in num_cols:
  
  # Остальной код без изменений
  q1 = df[column].quantile(0.25)
  q3 = df[column].quantile(0.75)
  iqr = q3 - q1
  
  lower_bound = q1 - (threshold * iqr) 
  upper_bound = q3 + (threshold * iqr)
   
  outliers = ((df[column] < lower_bound) | (df[column] > upper_bound)).sum()

  outliers_per_column[column] = outliers

# Вывод колонок с выбросами  
outliers_list = list(outliers_per_column.items())
outliers_list.sort(key=lambda x: x[1], reverse=True)

print('Колонки с выбросами:')   
for column, outliers in outliers_list:
  if outliers > 0:
      print(f'{column}: {outliers} выбросов')

Колонки с выбросами:
r2: 118 выбросов
A: 112 выбросов
C: 38 выбросов
mu: 38 выбросов
B: 21 выбросов
alpha: 5 выбросов


 выбросов не так много, просто удалим строчки с ними. Информации не потеряем

In [136]:

# Список для индексов строк с выбросами
outliers_indices = []

for column in num_cols:

  q1 = df[column].quantile(0.25)
  q3 = df[column].quantile(0.75)
  iqr = q3 - q1
  
  lower_bound = q1 - (threshold * iqr)
  upper_bound = q3 + (threshold * iqr)

  # Получаем индексы строк с выбросами
  outlier_indices = df[(df[column] < lower_bound) | (df[column] > upper_bound)].index
  
  # Добавляем их в общий список 
  outliers_indices.extend(outlier_indices)

# Удаляем дубликаты индексов
outliers_indices = list(set(outliers_indices))  

print(f'Найдено {len(outliers_indices)} строк для удаления')

# Удаляем строки с выбросами
df = df.drop(index=outliers_indices)

print('Датафрейм очищен от выбросов')

Найдено 306 строк для удаления
Датафрейм очищен от выбросов


На этом Outlier detection закончен.

In [137]:
# Сохранение в CSV-файл
df.to_csv('dataset_with_descriptors_without_errors_v2.2.csv', index=False)

# Duplicates handling

In [138]:
df = pd.read_csv('dataset_with_descriptors_without_errors_v2.2.csv')

Тут всё просто. Удаляем дубликаты по столбцу smiles

In [139]:
# Сохраняем исходное количество строк
initial_rows = len(df)

# Указываем столбец для поиска дубликатов
column_to_check = 'smiles'

# Удаляем дубликаты в указанном столбце и считаем количество удаленных строк
df = df.drop_duplicates(subset=[column_to_check], keep=False)
removed_rows = initial_rows - len(df)

# Выводим количество удаленных дубликатов
print(f"Удалено {removed_rows} дубликатов в столбце '{column_to_check}'.")

Удалено 8 дубликатов в столбце 'smiles'.


# Data normalization

Сначала я сокращу размерность удаляя столбцы с высокой корреляцией, а потом нормализую их.  
Нормализовать ДО - нельзя. Нормализация может привести к потере информации которая помогает искать корреляции.  

## Сокращение размерности путём удаления коррелирующих столбцов

In [140]:
# Фильтрация по коэффициенту корреляции
# Вычисление матрицы корреляции
corr_matrix = df.drop(['smiles', 'gap'], axis=1).corr()

# Выбор дескрипторов с корреляцией > 0.75 или < -0.75
high_corr_descriptors = set()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.75:
            high_corr_descriptors.add(corr_matrix.columns[i])

# Удаление выбранных дескрипторов, сохраняя столбцы 'smiles' и 'gap'
columns_to_drop = high_corr_descriptors - {'smiles', 'gap'}
df = df.drop(columns_to_drop, axis=1)

print("Удалены следующие дескрипторы с высокой корреляцией:")
print(columns_to_drop)

Удалены следующие дескрипторы с высокой корреляцией:
{'SpAD_Dzpe', 'VE3_Dzse', 'AATSC3se', 'SRW09', 'AATSC4v', 'ATS0Z', 'AATS0Z', 'ATSC6pe', 'GATS5p', 'bpol', 'ETA_eta_F', 'TIC5', 'SpAbs_DzZ', 'AATS2are', 'ATS6se', 'AATS1pe', 'AETA_eta', 'AMID_N', 'HeavyAtomCount', 'LogEE_Dzpe', 'ATS6s', 'RingCount', 'ZMIC5', 'SMR_VSA7.1', 'ATSC6i', 'VR2_Dzm', 'MATS4c', 'piPC5', 'SaaCH', 'SRW03', 'MPC8', 'ATS2are', 'Ipc', 'BCUT2D_CHGLO', 'AATSC5i', 'ETA_dEpsilon_A', 'TSRW10', 'ATS3se', 'BCUTs-1l', 'TPSA', 'ATSC3p', 'VE1_Dzare', 'AATS5s', 'MATS1are', 'MATS2i', 'ATS4are', 'GATS4dv', 'SMR_VSA6.1', 'VR3_A', 'BIC4', 'C1SP3', 'nO', 'BCUTv-1l', 'Xp-3dv', 'SMR_VSA5', 'AXp-3d', 'MATS5dv', 'SM1_Dzv', 'ATS3p', 'Chi4n', 'JGI2', 'AETA_eta_FL', 'BCUTare-1l', 'TIC3', 'ZMIC4', 'AATS4dv', 'LogEE_Dzm', 'BalabanJ', 'Xch-6dv', 'VSA_EState4.1', 'AATSC2s', 'VAdjMat', 'AATSC5c', 'TopoPSA', 'SpMAD_A', 'VR3_Dzare', 'FpDensityMorgan2', 'AATS4Z', 'AATSC1c', 'AATSC2pe', 'TMWC10', 'ATSC7Z', 'Xpc-4d', 'IC1', 'Diameter', 'ATS3pe', '

In [141]:
df

,smiles,A,B,mu,alpha,gap,zpve,u0,cv,MaxAbsEStateIndex,...,n6Ring,n3HRing,n4HRing,n6FRing,n7FRing,GGI4,GGI5,JGI4,Radius,TopoShapeIndex
0,N=C1OCC2=C1ON=C2,3.42254,1.67692,4.5643,67.77,0.2048,0.091971,-452.735916,24.110,7.091204,...,0,0,0,0,0,0.125000,0.000000,0.041667,2,1.000000
1,O=C1CCC11C2CCN12,3.00351,1.58602,1.9055,77.52,0.2091,0.147846,-401.909060,29.799,11.033750,...,0,1,1,0,0,0.222222,0.000000,0.055556,2,1.000000
2,O=CC1CCC2OCC12,3.11416,1.26756,1.6498,75.15,0.2215,0.160480,-423.030316,29.829,10.341435,...,0,0,1,0,1,0.276111,0.062500,0.069028,3,0.666667
3,C1C2C3N=C4OC1C4C23,3.12126,2.32028,3.6237,69.65,0.2066,0.127690,-400.695008,24.179,5.424769,...,0,0,1,0,0,0.000000,0.000000,0.000000,2,0.500000
4,CC1(O)C2CC1(O2)C#C,2.37047,1.54170,1.7895,76.02,0.2569,0.133262,-421.725719,33.112,9.414491,...,0,0,2,0,0,0.222222,0.000000,0.074074,2,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16493,c1coc2c1C(=O)CC2,3.17605,1.63047,3.0955,73.12,0.2127,0.115102,-420.687271,26.087,10.866620,...,0,0,0,0,0,0.125000,0.000000,0.041667,2,1.000000
16494,CC1CN1C(=N)C#N,3.12217,1.77668,3.1791,69.13,0.2075,0.118744,-358.830448,28.421,8.146759,...,0,1,0,0,0,0.205000,0.000000,0.051250,3,0.666667
16495,COC1CC(=N)OC1=O,3.40512,1.21702,2.1467,67.72,0.2416,0.123752,-475.063357,30.166,10.527685,...,0,0,0,0,0,0.205000,0.000000,0.041000,3,0.666667
16496,OC1CC11OCC1O,3.22232,1.74690,1.4181,60.90,0.2937,0.129128,-420.823950,29.183,8.923611,...,0,0,1,0,0,0.111111,0.000000,0.055556,2,1.000000


## Нормализация

In [142]:
from sklearn.preprocessing import MinMaxScaler

In [143]:
# Отбор числовых столбцов 
numeric_cols = df.select_dtypes(include=['float', 'int']).columns

# Создание скалера  
scaler = MinMaxScaler()

# Фитим скалер на числовых данных
scaler.fit(df[numeric_cols])  

# Нормализуем числовые столбцы
scaled_numeric_cols = scaler.transform(df[numeric_cols])

# Заменяем в датафрейме 
for i, col in enumerate(numeric_cols):
    df[col] = scaled_numeric_cols[:, i]
    
print(df[numeric_cols].describe())

                  A             B            mu         alpha           gap  \
count  16490.000000  16490.000000  16490.000000  16490.000000  16490.000000   
mean       0.296279      0.309778      0.279762      0.480009      0.507231   
std        0.168186      0.143438      0.150260      0.117493      0.140095   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.174441      0.205378      0.165397      0.403452      0.405237   
50%        0.259476      0.303495      0.259868      0.481045      0.499851   
75%        0.374548      0.402542      0.375910      0.558759      0.614698   
max        1.000000      1.000000      1.000000      1.000000      1.000000   

               zpve            u0            cv  MaxAbsEStateIndex  \
count  16490.000000  16490.000000  16490.000000       16490.000000   
mean       0.486717      0.547351      0.533570           0.594843   
std        0.152991      0.115433      0.137483           0.224188   
min     

In [145]:
# Сохранение в CSV-файл
df.to_csv('dataset_normalization__v2.3.csv', index=False)